In [1]:
import requests
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import json
import re

In [ ]:

# 以下程式碼會先將所有新聞文章網址爬下來
# url_1 = 'https://udn.com/search/word/2/%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E' # 新冠肺炎

print(datetime.now())

total_links = []

for i in range(75, 5375):
    # start with page 75, since the previous pages is all on 2021, end by 5375
    url = f'https://udn.com/api/more?page={i}&id=search:%E6%96%B0%E5%86%A0%E8%82%BA%E7%82%8E&channelId=2&type=searchword&last_page=5374'
    print(url)
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.9,zh-TW;q=0.8,zh;q=0.7',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
    }
    res = requests.get(url, headers=headers)
    json_str = json.loads(res.text)

    for i in range(0, len(json_str['lists'])):
        date = json_str['lists'][i]['time']['dateTime']
        date_formatter = "%Y-%m-%d %H:%M:%S"
        if (datetime.strptime(date, date_formatter) > datetime.strptime('2019-11-01', '%Y-%m-%d')) and \
                (datetime.strptime(date, date_formatter) < datetime.strptime('2021-01-01', '%Y-%m-%d')):
            if (json_str['lists'][i]['titleLink']) not in total_links:  # to connect original list after error happened
                total_links.append(json_str['lists'][i]['titleLink'])
    sleep_time = random.randint(6, 10)
    time.sleep(sleep_time)

print(datetime.now())

column = ['link']
df = pd.DataFrame(data = total_links, columns = column)
df.to_csv(r'./links.csv', index=0, encoding='utf-8-sig', header=None)


In [2]:
total_links =[]

with open(r'C:\Users\TibaMe\Desktop\TeamA\data\pyetl_udn_news_links_covid19.csv', 'r', encoding = 'utf-8-sig') as single_list :
    for each_line in single_list.readlines() :
        total_links.append(each_line.replace('\n', ''))
print(total_links[-5:])
print(len(total_links))

['https://udn.com/news/story/120944/4338169', 'https://opinion.udn.com/opinion/story/120902/4337076', 'https://udn.com/news/story/120940/4330412', 'https://health.udn.com/health/story/121019/4307194', 'https://ubrand.udn.com/ubrand/story/12117/4174194']
49570


In [3]:
total_links.index('https://ubrand.udn.com/ubrand/story/12117/4174194')

49569

In [ ]:

# 以下程式碼會將每個新聞網址裡所需的內容取出
# cut on 
total_list = []

for i in total_links[:49569] :
    print(total_links.index(i))
    headers = {'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9,zh-TW;q=0.8,zh;q=0.7', 'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
              }
    res = requests.get(i, headers=headers)
    soup = BeautifulSoup(res.text)
    single_list = []
    single_list.append('covid-19')
    try :
        single_list.append(soup.select_one('time[class="article-content__time"]').text)
        single_list.append(i)
        single_list.append(soup.select_one('h1[class="article-content__title"]').text)
        single_list.append(soup.select_one('div[class="article-content__paragraph"]').text.replace('\n','').replace('\r','').replace(' 跨年疫情新冠肺炎',''))
        total_list.append(single_list)
        columns = ['Key Word', 'Time', 'Website', 'News Article', 'Content']
        df = pd.DataFrame(data = total_list, columns = columns)
        df.to_csv(r'./content_covid_to49569.csv', index=0, encoding='utf-8-sig', header=True)
    except AttributeError :
        try :
            single_list.append(re.search(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}',soup.select_one('h1[class="story_art_title"]').text).group())
            single_list.append(i)
            single_list.append(soup.select_one('h1[class="story_art_title"]').text.split('分享')[0])
            content = ''
            for i in soup.select('p'):
                content += i.text
            single_list.append(content)
            total_list.append(single_list)
            columns = ['Key Word', 'Time', 'Website', 'News Article', 'Content']
            df = pd.DataFrame(data = total_list, columns = columns)
            df.to_csv(r'./content_covid_to49569.csv', index=0, encoding='utf-8-sig', header=True)
        except Exception as e :
            print(e)
    time_sleep = random.randint(6,10)
    time.sleep(time_sleep)
    print(single_list)



0
['covid-19', '2020-12-31 22:23', 'https://udn.com/news/story/120944/5137833', '新冠肺炎全球確診逾8267萬 超過180萬人病故', '法新社彙整官方數據顯示，截至格林威治標準時間31日11時（台灣31日晚間7時），全球至少180萬6072人死於2019冠狀病毒疾病（COVID-19，新冠肺炎），至少8267萬6050例確診。疫情從去年12月在中國爆發，迄今全球至少有5191萬8900人被認為已康復。法新社蒐集各國當局和世界衛生組織（WHO）提供的數據做出統計，但這恐怕只是實際感染總數的一部分，因為許多國家只檢測有症狀或症狀最嚴重的病例。全球30日通報新增1萬5599人病歿、67萬9925起確診病例。根據通報數據，新增病歿人數最多國家依序為美國（3927死）、巴西（1194死）與墨西哥（1052死）。美國仍是疫情最嚴重國家，累計34萬2414人病故、1974萬5137例確診。其他疫情嚴重國家還包括巴西（19萬3875人喪生、761萬9200例確診）、印度（14萬8738人喪生、1026萬6674例確診）、墨西哥（12萬4897人喪生、141萬3935例確診）及義大利（7萬3604人喪生、208萬3689例確診）。依照染疫死亡人數占人口比例，比利時是死亡率最高國家，每10萬人就有168人染疫喪命；接下來依序為斯洛維尼亞（128人）、波士尼亞與赫塞哥維納（123人）、義大利（122人）、北馬其頓共和國（119人）。從區域來看，歐洲累計56萬8862死、2630萬9例確診；拉丁美洲和加勒比海地區累計50萬5089死、1546萬5966例確診；美國和加拿大共計35萬7854死、2031萬6207例確診。亞洲共計21萬8751死、1386萬2834例確診；中東地區共計8萬9766死、396萬9128例確診；非洲共計6萬4805死、273萬865例確診；大洋洲共計945死、3萬1041例確診。疫情快報 ▪【最新數據】英國將建變種病毒平台 印尼染疫總數破百萬  ▪【數據圖表】新冠肺炎最終章何時登場 365天疫情數據解析  ▪【QA測驗】2020疫情肆虐 你的生活變什麼樣？ ▪【疫苗專題】全球新冠疫苗爭奪戰開跑 台灣起步早為何還落後？  ▪【疫情專題】新冠肺炎給出一記重拳 全球航空業到底有多慘？ 新冠

['covid-19', '2020-12-31 19:47', 'https://udn.com/news/story/7331/5137468', '對岸納疫苗優先族群  陸委會：勿視台人為試驗品', '陸方昨天宣布將在陸台籍民眾納入「優先安排」接種COVID-19疫苗對象。陸委會今天則提醒在陸民眾，應注意中國大陸疫苗風險，「也請陸方不要將台灣人民視為疫苗試驗的試驗品」。大陸委員會發言人邱垂正下午在例行記者會回應媒體相關提問時作上述表示。媒體追問，對岸官方稍早宣布，由中國國藥集團所研發的疫苗已獲有條件批准上市，政府是否仍不鼓勵在陸台籍民眾前往接種。邱垂正則表示，政府認為，應該等相關資訊更充分或有更多國際專業機構認證，讓民眾得到充分訊息後再審慎考量前往施打的必要性。他強調，2019冠狀病毒疾病（COVID-19，新冠肺炎）疫苗從研發到接種，都是屬於高度醫衛專業的事務，必須以嚴謹的科學程序來進行，以確保安全。邱垂正指出，陸委會提醒在中國大陸的台灣民眾，注意陸製疫苗風險資訊，審慎考量現階段施打的必要性，「完全是出於保障民眾健康安全的提醒」，反觀陸方一再以不專業態度，將防疫事務與政治宣傳掛高，藉故批評台灣，相信各界及台灣民眾都不會接受，「也請陸方不要將台灣人民視為疫苗試驗的試驗品」。媒體詢及中國國務院台灣事務辦公室發言人朱鳳蓮昨天稱，使用「武漢肺炎」來稱呼疫情具有歧視性。邱垂正指出，針對中國大陸在2019年底傳出的這一波疫情，國際間都有不同的稱呼，台灣主管機關所訂的法定傳染病名稱為「嚴重特殊傳染性肺炎」。邱垂正強調，各地對於流行病的命名或簡稱，是基於學理定義或溝通便利所慣用、沿用的稱呼，不一定都具有負面的意義。台灣是一個言論自由的社會，政府對於各界如何稱呼這波疫情，從未加以提倡也沒有限制，「根本不存在陸方所稱汙名化的問題」。邱垂正指出，善盡防疫職責，維護世人的健康安全，才是負責任的政府，守護人民的重中之重，至於用什麼字眼來稱呼疫情，與防疫無關，「從來不是我方認為需要考慮的事」。 疫苗邱垂正疫情']
12
['covid-19', '2020-12-31 19:44', 'https://udn.com/news/story/121903/5137508', '北市跨年19時逾4000人進場 秩序良好觀眾自制力佳', '北市跨年晚會因新冠肺炎英國變種病毒入境，管制人數從8

['covid-19', '2020-12-31 18:01', 'https://udn.com/news/story/121903/5137329', '義大世界跨年會場備妥5千張椅子 晚間8點開始直播', '義大世界跨年夜施放十分十秒跨年焰火，象徵成立十周年十全十美。義大世界指出，今晚8點跨年晚會登場，藝人楊乃文、楊丞琳、「情歌王子」Bii畢書盡等人在跨年前後壓軸演出。因疫情關係，跨年晚會改成線上直播，只允許義大皇家酒店、義大天悅飯店、跨年住房套裝專案房客，以及義大別墅社區住戶採實名制參加跨年晚會。義大世界跨年晚會只開放有資格的飯店住客與別墅區住戶進入會場，總數不超過5千人。現場擺放5千張椅子、間距1.6公尺，民眾須持義大世界印證的實名票才能入場，民眾須量體溫、戴口罩，除喝水外禁止飲食，入口處有專人檢查入場券。義大世界今凌晨宣布，因應COVID-19疫情，「2021紫耀義大 享樂拾光跨年晚會」改為線上直播，一般民眾及已取得VIP觀賞區入場券的民眾不得入場。晚間8點開始，華視主頻、華視新聞資訊台、華視YouTube頻道轉播，直播晚會全程，現場演唱歌手將與電視機前民眾一同迎接2021年。']
23
['covid-19', '2020-12-31 17:57', 'https://udn.com/news/story/11091/5137324', '【重磅快評】台灣會變成新冠疫苗白老鼠嗎？', '衛福部長陳時中「終於」宣布我國已經取得的新冠疫苗，是英國的牛津/阿斯特捷利康(AZ)疫苗，共1000萬劑。不過相關細節，還是沒有交代清楚。如陳時中說「最快」3月可取得，卻沒說「最慢」何時可以拿到；另外，對於其他疫苗，陳時中則以「防止政治打壓」拒絕透露細節；但從提到的「冷鏈設備建置」來看，另一支是當初「得而復失」的BNT/輝瑞疫苗的可能性極高。但最重要的問題仍然是，AZ疫苗到現在都並未獲得美國和歐盟的授權，台灣會不會因此成為疫苗的白老鼠？在整個防疫和疫苗取得上，台灣現在顯然面臨一個尷尬的局面：11月初衛福部以冷鏈、授權協議等問題百般刁難BNT疫苗導致採購破局；儘管本土感染案例不多，但在境外變種病毒案案例逐漸增多情況下，疫情隨時可能有發生破口的危險，又迫使衛福部不得不加快疫苗採購腳步。COVAX機制無法掌控，AZ疫苗就成了唯一可能的救命稻草。問題是，AZ疫苗在三期臨床階段一

['covid-19', '2020-12-31 16:23', 'https://udn.com/news/story/120940/5137088', '變種病毒恐仍入台？陳時中：國內防疫管理已上軌道', '英國變種病毒株傳播力增強，雖昨日公布自元旦起實施邊境管制政策，但指揮中心指揮官陳時中認為，變種病毒有可能入台，然國內防疫已與第一波武漢疫情時不同，現在管理已上軌道。非本國籍者若沒有居留證，得依有商務履約、有配偶及未成年子女或是專案許可入境。指揮中心指揮官陳時中表示，以前專案授權給各地方，現在改為給指揮中心核准，把病毒與社區隔開。有些地方政府已在辦理，指揮中心態度公開。陳時中表示，武漢當時我們都籠罩於恐懼中，因為恐懼來源就是武漢封城以及小區的報導，鐵鍊鎖起來，木板等，建立方倉醫院，我們沒有做到最高規格，但現在經過11個月，對於認知就不一樣了。如今新冠疫情再起一波，英國變種病毒株已入侵約50個國家，陳時中表示，現在防疫能量有所不同，加上歐洲沒有管控與英國之間的交通，新型病毒傳染力高，仍有可能進來。陳時中表示，英國算是民主國家的先驅，揭露度很高，因此我國防疫措施沒有很大的不一樣。英國現在沒有封城，所以我們減班，武漢他們封城，我們沒辦法飛，所以我們積極協調包機，所以並沒有對武漢非常嚴；兩波疫情都用一樣的方式，流程都一樣，集中檢疫完等採檢，集中管理，集中檢疫也沒有群聚或是脫逃。然新冠病毒變異到現在已有20幾次，陳時中表示，英國透露這次變異株的訊息更清楚，四分之一感染此病毒，到現在三分之二，仍誠實對世界宣告。陳時中補充，英國公佈宣布這樣的訊息會對他們有影響，但他們還是願意做為負責任的世界公民，所以認為英國是認真負責的態度。疫情快報 ▪【最新數據】英國將建變種病毒平台 印尼染疫總數破百萬  ▪【數據圖表】新冠肺炎最終章何時登場 365天疫情數據解析  ▪【QA測驗】2020疫情肆虐 你的生活變什麼樣？ ▪【疫苗專題】全球新冠疫苗爭奪戰開跑 台灣起步早為何還落後？  ▪【疫情專題】新冠肺炎給出一記重拳 全球航空業到底有多慘？ 新冠肺炎COVID-19變種病毒陳時中']
35
['covid-19', '2020-12-31 16:18', 'https://udn.com/news/story/7238/5137048', '瀚亞攜手伊甸基金會 與失能家庭共同抗疫', '2

['covid-19', '2020-12-31 15:32', 'https://udn.com/news/story/120944/5136362', 'Q&A／英國新冠變種病毒擴散全球 專家籲：切勿驚慌', '英國出現變種新冠病毒並且擴散至世界各地，科學家火速針對變種病毒研究，公衛專家也呼籲民眾鎮靜以對，切勿恐慌。變種新冠病毒在疫情初期於中國爆發時就已存在；病毒在複製或感染過程中，常出現變異或些微改變。波士頓學院全球衛生計畫主任蘭崔根(Philip Landrigan)說：「大部分的改變太過細微，並不會影響病毒產生致病的能力。」較讓人擔心的是，病毒改變表面的蛋白質，幫助其逃過免疫系統或是藥物；但病毒若要徹底改變，需要極大變動。變種病毒如何變得強大？若其中一種變種病毒在「超級傳播者」的大型活動中擴散，或變異出特定優勢，那將更容易傳播。科學家仍在確認英國變種病毒是否更容易傳染，但目前已出現一些證據顯示，答案是肯定的。蘭崔根說：「某一種變種病毒感染速度更快，傳染給更多人，贏了與其他變種的競爭。」變種病毒為何讓人擔心？變種病毒有接近20多種變異，劍橋大學病毒專家古普塔(Ravi Gupta)博士表示，變種病毒的傳染力，可能比最常見的新冠病毒強上兩倍，但此研究尚未獲得複審或正式於期刊上發表。變種病毒死亡率更高嗎？目前並未有明確證據證實此說法，但科學家正密切關注；不過世界衛生組織(WHO)表示，變種病毒在病情嚴重程度上，和原先的病毒差異不大。變種病毒會影響治療嗎？目前科學家也在研究，檢視原有藥物對變種病毒的效力；不過禮來藥廠(Eli Lilly)表示，實驗結果顯示其藥物對變種病毒依舊有效。科學家相信現有疫苗仍能抵禦變種病毒，但仍在作實驗。英國官員也重申，目前沒有資料顯示變種病毒能削弱疫苗效力；疫苗能促使整個免疫系產生反應，而不僅是單純產生抗體，因此目前評估仍舊有效。疫情快報 ▪【最新數據】英國將建變種病毒平台 印尼染疫總數破百萬  ▪【數據圖表】新冠肺炎最終章何時登場 365天疫情數據解析  ▪【QA測驗】2020疫情肆虐 你的生活變什麼樣？ ▪【疫苗專題】全球新冠疫苗爭奪戰開跑 台灣起步早為何還落後？  ▪【疫情專題】新冠肺炎給出一記重拳 全球航空業到底有多慘？ 新冠肺炎COVID-19疫苗變種病毒']
44
['covid-19', '2020-12-31 15:26

...

['covid-19', '2020-12-27 15:22', 'https://udn.com/news/story/120944/5125298', '歐盟疫苗計畫啟動！德國、匈牙利、斯洛伐克偷偷搶先打', '全球歷史上規模最大的疫苗接種運動已經開始，根據彭博社收集的數據，包括美國在內的七個國家民眾已經接種了超過420萬劑新冠疫苗，而歐盟也帶在27日開始接種計畫，不過其分發部署計畫卻也因此被破壞。德國，匈牙利及斯洛伐克26日在收到第一批新冠疫苗僅數小時，即開始為首批人士接種，成為歐盟首先接種國家，不過卻破壞了歐盟27個成員國原定於27日協調接種的計畫。位於德國東北薩克森-安哈爾特州(Saxony-Anhalt)一家護理院，一名101歲院友科沃扎拉(Edith Kwoizalla)成為德國接種新冠疫苗第一人；德國衛生部長斯潘（Jens Spahn）稱是聖誕節的好消息，是恢復生活的關鍵。匈牙利首者布達佩斯南部的中央醫院，首先為醫護人員接種；至於斯洛伐克第一個接種的，是尼特拉市的大學醫院60歲傳染病高級專家科默里（Vladimir Krcmery），該國衛生部長卡賈奇(Marek Krajci)形容是歷史時刻。儘管這三個國家先插頭炷香，但歐盟主席烏蘇拉．馮德萊恩（Ursula von der Leyen）發布視頻，強調疫苗向近4.5億人口推出，是一個動人的團結時刻。她表示在經歷艱辛的一年後翻開新一頁，新冠疫苗已交付所有歐盟國家，明天(27日)在整個歐盟範圍內接種。目前歐盟至少有1600萬宗確診病例，33萬6000人死亡，由於英國、美國及加拿大率先接種德國製藥公司BioNTech的疫苗，讓歐盟政客們都感到沮喪，但現在總算鬆一口氣。法國雖然仍在注視疫苗是否存在潛在問題，但已決定長者獲優先接種。西班牙首批疫苗已運抵中部城市瓜達拉哈拉(Guadalajara)，首先為療養院的院友接種。義大利新冠死亡人數是7萬1000，是歐盟國家之冠，首都羅馬Spallanzani傳染病專科醫院的護士首先接種。波蘭首都華沙內政部醫院，一名醫生及一名護士是該國首先接種的首兩人。歐盟各國首批獲得的疫苗數量都在1萬劑以下，預計到下月歐盟才能正式展開大規模疫苗接種計畫。目前法國及西班牙均發現在英國傳播的病毒變種，但BioNTech有信心其新冠疫苗可以對抗新的變種。▌延伸推薦：全球各國接種疫苗時程出爐

['covid-19', '2020-12-27 15:03', 'https://udn.com/news/story/120940/5125553', '疫調足跡蓋牌？疫調人員：有名人、酒店等關鍵字未公布', '被解雇的紐西蘭籍長榮航空機師染疫，中央流行疫情指揮中心僅公布天母SOGO等幾處疫調足跡；隔天台北市議員徐巧芯拿出隱藏版的疫調足跡，指該機師還去過日本交流協會、台北火車站等地，引發爭議。基層疫調人員說，過去確實發生過企業名人在酒店等敏感地點的疫調足跡未公布，但一切以指揮中心的說法為準，通常沒公布的，就是被指揮中心判定「情況已掌握，無傳播疑慮」。回顧今年四月間爆發的海軍磐石艦群聚事件，高雄市衛生局前後總共公布六波確診個案足跡共卅九處，地點涵蓋大賣場、百貨公司、汽車旅館等，但在第一波足跡公布時，網路同時也流傳一分更詳細的足跡圖，多家餐廳、火鍋店名列其中，衛生局一度遭質疑是為了保護業者有「蓋牌」之嫌，引發爭議。疫調人員指出，疫調足跡會先內部審核評估，若無法掌握個案當時的活動地點有多少人或哪些特定對象，才會公布出去，提醒到過該處的民眾注意；此外，有官兵是在社區活動後多日才出現症狀，考量傳播風險較低，只要求足跡場所加強清潔消毒，足跡不會公布。北部疫調人員則說，基層做疫調都會盡量詳盡，有過確診者到過某家酒店，事後調查足跡時發現某些企業名人和官員也曾逗留，報給指揮中心後，這些個資和足跡並未公布。追蹤深度內容：疫調足跡標準何在？ 為何紐籍機師僅三言兩語、移工卻鉅細靡遺疫情快報 ▪【最新數據】印尼染疫總數破百萬 醫療系統瀕臨崩潰  ▪【數據圖表】新冠肺炎最終章何時登場 365天疫情數據解析  ▪【QA測驗】2020疫情肆虐 你的生活變什麼樣？ ▪【疫苗專題】全球新冠疫苗爭奪戰開跑 台灣起步早為何還落後？  ▪【疫情專題】新冠肺炎給出一記重拳 全球航空業到底有多慘？ 新冠肺炎COVID-19紐西蘭機師']
633
'NoneType' object has no attribute 'group'
['covid-19']
634
'NoneType' object has no attribute 'group'
['covid-19']
635
['covid-19', '2020-12-27 14:59', 'https://udn.com/news/story/681

['covid-19', '2020-12-27 13:28', 'https://udn.com/news/story/6809/5125272', '15駕駛染疫無法出勤 東京都營地鐵車班減3成', '日本2019冠狀病毒疾病（COVID-19）疫情延燒，其中以東京都最為嚴峻，連都營地下鐵大江戶線都有15名駕駛染疫無法出勤，造成車班被迫減到原本的7成左右，都政府也出面致歉。日本放送協會（NHK）報導，東京都政府經營的地下鐵大江戶線，由於電車駕駛相繼確診染疫，被迫從今天開始減班到原本的7成左右。東京都政府交通局表示，從本月15日以來，都營地下鐵大江戶線屬於江東區清澄乘務段的15名駕駛，相繼感染2019冠狀病毒疾病。被視為確診者密切接觸者的同事，目前也都在家隔離，因此，清澄乘務段目前無法工作的駕駛有21人，約占整體約一成。由於無法照一般的勤務配置人力運行，造成大江戶線行駛車班將從本月27日起到2021年1月11日止，減少到原本的約7成，未來將視駕駛健康恢復狀況，逐步恢復到正常車班。東京都政府對造成乘客不便表達歉意。 京都東京日本']
646
['covid-19', '2020-12-27 12:35', 'https://udn.com/news/story/120940/5125208', '旅英4千留學生將放寒假 教育部加強返台協助機制', '英國近日傳出感染變種新冠病毒病例。教育部表示，我國在英國有4050名留學生，目前英國交通順暢，移動不受阻。教育部尊重留學生返台意願，欲返台的學生若有緊急困難，將由駐英代表處或駐英教育組聯繫協助，盡力提供安排。教育部國際司表示，歐美各大學將放寒假，教育部尊重海外留學生返國意願，並將透過駐外機構，加強既有急難救助機制，提供留學生及時、適當的協助。教育部也提醒留學生注意英國政府各項防疫措施和相關規定，並請54個台灣同學會會長轉達留學生各項資訊。教育部引述英國政府公告，近期英國東南方疫情持續加劇，英國政府12月19日緊急發布，將原本的中等（medium）、高等（high）、極高（very high）三階段防疫措施之外，增加更緊縮的第四級措施。駐英教育組也同步在臉書提醒留學生配合英國政府防疫規定。一名林姓學生正在倫敦攻讀博士，他表示英國近日疫情加劇，英國政府昨天宣布更嚴格的防疫措施，民眾不能去服飾店、禮物店等非必要店家，街上只剩超

['covid-19', '2020-12-27 10:27', 'https://udn.com/news/story/121687/5125022', '拜登警告 川普若續擋紓困將導致毀滅性後果', '美國總統當選人拜登（Joe Biden）今天警告，如果已獲國會通過的2019冠狀病毒疾病（COVID-19, 新冠肺炎）紓困法案繼續被總統川普拖延簽署，將出現「毀滅性的後果」。拜登是在川普讓各界期待已久的紓困出現變數、數以百萬計美國人看著失業津貼到期之後，公開發表上述看法。川普要求議員提高直接發給美國納稅人的金額，從法案規定的600美元，增為3倍以上。拜登在聲明中說：「這種放棄責任的作法會產生毀滅性的後果，今天，約有1000萬美國人會失去失業救濟金。」這項紓困措施和1.4兆美元政府支出法案包裹投票通過，而如果1.4兆美元的預算法案無法生效，聯邦政府28日午夜就得被迫關門。拜登還說：「短短數天之內，政府經費就會用罄，重要服務和軍方人員的薪資將岌岌可危。」「不到一週內，暫緩迫遷令（evictionmoratorium）就會到期，恐讓數以百萬人在假期期間被逐出家門。」 美國大選川普拜登']
658
['covid-19', '2020-12-27 10:11', 'https://udn.com/news/story/120940/5125015', '境外移入+2！ 今晚英國返台120人 指揮中心下午2時說明', '繼昨日新增3例境外確定病例，中央流行疫情指揮中心今天宣布，再增2例新冠肺炎境外移入個案，指揮中心將於下午2時舉行臨時記者會，由發言人莊人祥對外說明，屆時沒有直播。英國變種新冠肺炎病毒於全球肆虐，今晚將有120名旅客自英國搭乘華航班機抵台，這是變種病毒現蹤之後，單日自英國入境最高人數。指揮中心表示，防疫採取最高規格，返台者進入集中檢疫所14天，入境後及出關前都要採檢。疫情快報 ▪【最新數據】印尼染疫總數破百萬 醫療系統瀕臨崩潰  ▪【數據圖表】新冠肺炎最終章何時登場 365天疫情數據解析  ▪【QA測驗】2020疫情肆虐 你的生活變什麼樣？ ▪【疫苗專題】全球新冠疫苗爭奪戰開跑 台灣起步早為何還落後？  ▪【疫情專題】新冠肺炎給出一記重拳 全球航空業到底有多慘？ 新冠肺炎COVID-19']
659
['covid-19', '2020-12-27 09:3

['covid-19', '2020-12-27 06:32', 'https://udn.com/news/story/120944/5124437', '圖表看疫情／全球染疫破8000萬人 日本疫情正持續擴大', '疫情快報 ▪【最新數據】印尼染疫總數破百萬 醫療系統瀕臨崩潰  ▪【數據圖表】新冠肺炎最終章何時登場 365天疫情數據解析  ▪【QA測驗】2020疫情肆虐 你的生活變什麼樣？ ▪【疫苗專題】全球新冠疫苗爭奪戰開跑 台灣起步早為何還落後？  ▪【疫情專題】新冠肺炎給出一記重拳 全球航空業到底有多慘？ 新冠肺炎COVID-19']
666
['covid-19', '2020-12-27 06:03', 'https://udn.com/news/story/120944/5124866', '全球染疫人數突破8000萬  逾175萬人不治', '約翰霍普金斯大學統計顯示，繼耶誕節新增47萬2000人染疫後，全球各地確診感染2019冠狀病毒疾病（COVID-19）人數於26日早上突破8000萬大關，累計病故增至逾175萬人。約翰霍普金斯大學（Johns Hopkins University）統計顯示，截至台灣時間27日凌晨3時，全球各地確診感染增至8008萬1092例，累計染疫病故攀升至175萬4091人。美國仍是全球疫情最嚴重國家，累計通報確診1880萬165例，其後依序是印度通報確診1016萬9118例、巴西744萬8560例。美國境內染疫不治33萬678人，排名全球最多，其次是巴西19萬488死以及印度14萬7343死。疫情快報 ▪【最新數據】印尼染疫總數破百萬 醫療系統瀕臨崩潰  ▪【數據圖表】新冠肺炎最終章何時登場 365天疫情數據解析  ▪【QA測驗】2020疫情肆虐 你的生活變什麼樣？ ▪【疫苗專題】全球新冠疫苗爭奪戰開跑 台灣起步早為何還落後？  ▪【疫情專題】新冠肺炎給出一記重拳 全球航空業到底有多慘？ 新冠肺炎COVID-19']
667
['covid-19', '2020-12-27 06:03', 'https://udn.com/news/story/120944/5124891', '法國新增3093人染疫  較耶誕節前兩天大幅降低', '法國衛生當局今天通報，過去24小時新增3093人確診感染2019冠狀病毒疾病（COV

In [ ]:
columns = ['Key Word', 'Time', 'Website', 'News Article', 'Content']
df = pd.DataFrame(data = total_list, columns = columns)
df.to_csv(r'./content_新冠肺炎_7303to12445.csv', index=0, encoding='utf-8-sig', header=True)